# Corrector de textos (v 1.0)

Las siguientes funciones corrigen una cadena de texto conectándose a la API de un diccionario de español. En las siguientes versiones debemos pensar cómo reducir el tiempo de compilación. ¿Mejorará con una conexión a internet más rápida? Debemos probarlo con diferentes conexiones. 

In [25]:
import time
import lang_pytools
import pandas as pd
import numpy as np
from multiprocessing import Pool
from lang_pytools import corrector
from requests_html import HTMLSession

# Catálogo medicinas

In [26]:
urls="https://www.farmaciasanpablo.com.mx/medicamentos/antimicoticos/c/06001"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/dermatologicos/c/06002"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/dolor/c/06003"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/especialidades-medicas/c/06004"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/estomacal/c/06005"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/genericos/c/06006"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/gripe-y-tos/c/06007"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/gripe-y-tos/c/06007?q=%3Arelevance&page=1"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/gripe-y-tos/c/06007?q=%3Arelevance&page=2"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/gripe-y-tos/c/06007?q=%3Arelevance&page=3"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/gripe-y-tos/c/06007?q=%3Arelevance&page=4"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/gripe-y-tos/c/06007?q=%3Arelevance&page=5"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/gripe-y-tos/c/06007?q=%3Arelevance&page=6"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/gripe-y-tos/c/06007?q=%3Arelevance&page=7"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/oftalmicos/c/06008"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/pruebas-antidoping/c/06009"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/relajantes/c/06010"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/supervision-medica/c/06011"
urls+=" https://www.farmaciasanpablo.com.mx/medicamentos/varices/c/06012"
urls=urls.split(" ")

def catalogo_medicinas(url,cat_medicamentos):
    '''
    Esta función recopila los nombres de los medicamentos disponibles 
    en la farmacia SP con el fin de tener un catálogo de medicinas. 
    usar sleep() para no ser bloqueado temporalmente de la página.
    '''
    with HTMLSession() as sess:
        r = sess.get(url)
    tablas=r.html.find("div[class='item']")
    for tab in tablas:
        med=tab.text[0:tab.text.find(' ')].lower().replace("-","")
        cat_medicamentos.append(med)
    cat_medicamentos=list(np.unique(cat_medicamentos))
    time.sleep(5)
    return cat_medicamentos

def cat_medicamentos():
    cat_med=[]
    for url in urls:
        cat_med=catalogo_medicinas(url,cat_med)
    return cat_med

In [27]:
if input("Crear catálogo? (Sí: 0, No: 1) ") == '1':    
    medicamentos=cat_medicamentos()
    medicamentos=[m.replace("\n"," ") for m in medicamentos]
    medicamentos.append("toraco")
    medicamentos.append("colocutanea")

## Comprobación ortográfica de campo de texto (ejemplo SEGSAL)

In [39]:
%%time

df_acr=pd.read_excel(r"C:\Users\AN012491\Desktop\Alteryx Segsal\npl_causa raiz_v2.0.xlsx")
df_heridas=pd.read_excel(r"C:\Users\AN012491\Desktop\Alteryx Segsal\npl_heridas_v2.0.xlsx")
df_actividades=pd.read_excel(r"C:\Users\AN012491\Desktop\Alteryx Segsal\npl_actividad_v2.0.xlsx")

lpal_acr=list(df_acr['palabras'])
lpal_heridas=list(df_heridas['palabras'])
lpal_actividades=list(df_actividades['palabras'])

if __name__=="__main__":
    
#     if input("Crear catálogo: (0=Si, 1=No) ")=='0':
#         medicamentos=cat_medicamentos()
#         medicamentos=[m.replace("\n"," ") for m in medicamentos]
    
#     medicamentos=medicamentos.append("toraco")
#     medicamentos=medicamentos.append("colocutanea")
    lpals=[lpal_acr,lpal_heridas,lpal_actividades]
    num_processors = 6
    p=Pool(processes = num_processors)
    L=[]

    for lpal2 in lpals:
        lpal=[[x,medicamentos] for x in lpal2]
        output = p.map(corrector,lpal)
        L.append(output)

Wall time: 6min 58s


In [40]:
from pandas import ExcelWriter

df_acr['palabras']=L[0]
df_heridas['palabras']=L[1]
df_actividades['palabras']=L[2]

with ExcelWriter('procesamiento_lenguaje.xlsx') as writer:
    df_acr.to_excel(writer, sheet_name='ACR', index = False)
    df_heridas.to_excel(writer, sheet_name='heridas', index = False)
    df_actividades.to_excel(writer, sheet_name='actividad', index = False)